In [ ]:
# ALGO = "cosine/all-MiniLM-L6-v2"
# ALGO = "cosine/all-mpnet-base-v2"
ALGO = "cosine/distiluse-base-multilingual-cased-v1"
# ALGO = "cross-encoder/ms-marco-MiniLM-L12-v2"
# ALGO = "cross-encoder/stsb-roberta-large"
# ALGO = "dot/msmarco-bert-base-dot-v5"

TEST = "DETECT_QUANTIZATION_SPOOF"

In [ ]:
import os, json
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import Markdown, display_markdown
from perform_embedding_based_replication_v1 import (
    compute_scores,
    make_sim_callback,
    create_data_subsets,
    plot_roc_curve,
    plot_score_histogram,
    calculate_roc_metrics,
    write_summary,
)
import numpy as np

from argparse import Namespace

N = None
sim_callback = make_sim_callback(ALGO)

df = []
replications_dir = os.path.join("..", "replications")
for filename in os.listdir(replications_dir):
    filepath = os.path.join(replications_dir, filename)
    data = json.load(open(filepath, "r"))
    compute_scores(N, sim_callback, data, df)
df = pd.DataFrame(df)
df["count"] = 1

In [ ]:
# "meta-llama/Llama-3.1-8B-Instruct" "context-labs/neuralmagic-llama-3.1-8b-instruct-FP8" "meta-llama/Llama-3.2-3B-Instruct"

_3_1_8B_FP16 = "meta-llama/Llama-3.1-8B-Instruct"
_3_1_8B_FP8 = "context-labs/neuralmagic-llama-3.1-8b-instruct-FP8"
_3_2_3B_MODEL = "meta-llama/Llama-3.2-3B-Instruct"

DIFFERENT_MACHINE_TEST = (df["inference_machine"] == "4090") & (
    df["replication_machine"] == "3090"
)  #
DIFFERENT_QUANTIZATION_TEST = df["original_model"].isin(
    [_3_1_8B_FP16, _3_1_8B_FP8]
) & df["replication_model"].isin([_3_1_8B_FP16])
DIFFERENT_MODEL_TEST = (df["original_model"].isin([_3_1_8B_FP16, _3_2_3B_MODEL])) & (
    df["replication_model"].isin([_3_1_8B_FP16])
)

if TEST == "DETECT_QUANTIZATION_SPOOF":
    name = "Detect Quantization Spoofing"
    subset_df = df[DIFFERENT_MACHINE_TEST & DIFFERENT_QUANTIZATION_TEST]
elif TEST == "DETECT_MODEL_SPOOF":
    name = "Detect Model Spoofing"
    subset_df = df[DIFFERENT_MACHINE_TEST & DIFFERENT_MODEL_TEST]
else:
    raise Exception(f"Unknown TEST: {TEST}")


import importlib
import perform_embedding_based_replication_v1 as mylib

importlib.reload(mylib)

display(Markdown(f"{name} / {len(subset_df)}"))
score_column = "similarity"

selected_threshold = None

fig, ax = plt.subplots()
mylib.plot_score_histogram(
    subset_df, score_column, name, ax=ax, selected_threshold=selected_threshold
)
plt.show()

mylib.plot_roc_curve(df, score_column)
plt.show()

summary = mylib.write_summary(subset_df, score_column)
print(summary)